In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
from sklearn.linear_model import LogisticRegression
stop_wordsR = set(stopwords.words('russian'))
stop_wordsE = set(stopwords.words('english'))


train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

X_train = train_df["title"].values
X_test = test_df["title"].values
y_train = train_df["label"].values

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 666.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=b407715ba8b54af21039ae3b2de214aa753fbb0fdf9c7a7d79195a8d7f5a92db
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [60]:
train_df.head(30)
print(y_train.sum())

16715


In [ ]:
#!pip install FastText

Векторизация

In [2]:
text=np.copy(X_train)

y_tr=np.copy(y_train)
urld = train_df["url"].values

text=np.where(text==None,'0',text)
text=[str(i) for i in text]


def Stemming(data,stemmer):
    stemmed_data = [' '.join([stemmer.stem(word) for word in sentence.split() if ((word not in stop_wordsR) or (word not in stop_wordsE))]) for sentence in data]
    return stemmed_data# or (word not in string.punctuation)

def tfidf_vectorize(data, tfidf):
    X = tfidf.fit_transform(data)
    return X

morph = MorphAnalyzer()

def lemmatize_text(text):
    lemmatized_tokens = []
    for sentence in text:
        tokens = word_tokenize(sentence)
        lemmatized_tokens.append(' '.join([morph.parse(token)[0].normal_form for token in tokens]))
    return lemmatized_tokens

def word2vec_vectorize(data):
    model = Word2Vec(data, window=5, min_count=1, workers=4)
    vectorized_data = np.array([np.mean([model.wv[token] for token in sentence], axis=0) for sentence in data])
    return vectorized_data

def fast_text(texts):
    # Преобразование строк в формат, подходящий для FastText
    with open('data.txt', 'w', encoding='utf-8') as f:
        for text in texts:
            f.write(f'__label__1 {text}\n')

    # Обучение FastText модели
    model = fasttext.train_supervised('data.txt')

    # Получение векторов для каждой строки
    vectors = [model.get_sentence_vector(text) for text in texts]
    return vectors


st1=SnowballStemmer("russian")
st2=SnowballStemmer("english")

for i in range(len(urld)):
  urld[i]=urld[i].replace('.',' ')


stem_data=Stemming(text,st1)
stem_url=Stemming(urld,st2)

#vectorised=fast_text(stem_data)
#vectorised_url=fast_text(stem_url)

#stem_data=lemmatize_text(text)
#stem_url=lemmatize_text(urld)

tfidf1 = TfidfVectorizer(ngram_range=(1,2))
tfidf2 = TfidfVectorizer(ngram_range=(1,2))

vectorised=tfidf_vectorize(stem_data,tfidf1)
vectorised_url=tfidf_vectorize(stem_url,tfidf2)

#vectorised=word2vec_vectorize(stem_data)
#vectorised_url=word2vec_vectorize(stem_url)

Классификация

In [23]:
model=MultinomialNB()
model_url=MultinomialNB()
model.fit(vectorised,y_tr)
model_url.fit(vectorised_url,y_tr)
y_pred=model.predict_proba(vectorised)[:,0]
y_pred_url=model_url.predict_proba(vectorised_url)[:,0]

#Попытка применить вторую модель на предсказаниях двух
mod_gen=LogisticRegression()
mas=np.vstack((y_pred, y_pred_url)).T
print(mas.shape)
mod_gen.fit(mas,y_tr)
pr=np.where(mod_gen.predict_proba(mas)>0.65,0,1)[:,0]
print(f1_score(pr,y_tr))
print(confusion_matrix(pr,y_tr))

y_gen=np.where(((y_pred+y_pred_url)/2)>0.71, 0,1)
#print(f1_score(y_tr, y_pred))
#print(f1_score(y_tr, y_pred_url))
print(f1_score(y_tr, y_gen))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_gen,y_tr))

(135309, 2)
0.9814385848801178
[[118380    403]
 [   214  16312]]
0.981243059932175
[[118336    367]
 [   258  16348]]


Submit

In [26]:
X_test_title=test_df['title']
urld=test_df['url']
#for i in range(len(urld)):
  #urld[i]=urld[i].replace('.',' ')
stem_datap=Stemming(X_test_title,st1)
stem_urlp=Stemming(urld,st2)

vectorisedp=tfidf1.transform(stem_datap)
vectorised_urlp=tfidf2.transform(stem_urlp)

y_pred=model.predict_proba(vectorisedp)[:,0]
y_pred_url=model_url.predict_proba(vectorised_urlp)[:,0]

y_gen=np.where(((y_pred+y_pred_url)/2)>0.7, 0,1)
#y_gen=np.where(((6*y_pred/5+4*y_pred_url/5)/2)>0.78, 0,1)
#mas=np.vstack((y_pred, y_pred_url)).T
#y_gen=np.where(mod_gen.predict_proba(mas)>0.65,0,1)[:,0]

test_df["label"] = y_gen

test_df[["ID", "label"]].to_csv("ml_baseline.csv", index=False)

!cat ml_baseline.csv | head

ID,label
135309,0
135310,0
135311,0
135312,1
135313,0
135314,0
135315,0
135316,0
135317,0


Попытка понижения размерности

In [ ]:

from sklearn.decomposition import TruncatedSVD

tsvd = TruncatedSVD(n_components=10)

# apply the truncatedSVD function
X_sparse_tsvd = tsvd.fit(vectorised).transform(vectorised)
print(X_sparse_tsvd)

model=LogisticRegression()
model_url=MultinomialNB()
model.fit(X_sparse_tsvd,y_tr)
model_url.fit(vectorised_url,y_tr)
y_pred=model.predict_proba(X_sparse_tsvd)[:,0]
y_pred_url=model_url.predict_proba(vectorised_url)[:,0]
#model_gen=RandomForestClassifier()
#model_gen.fit(y_pred,y_pred_url,y_tr)
#print(f1_score(model_gen.predict(y_pred,y_pred_url),y_tr))

y_gen=np.where(((y_pred+y_pred_url)/2)>0.68, 0,1)
#print(f1_score(y_tr, y_pred))
#print(f1_score(y_tr, y_pred_url))
print(f1_score(y_tr, y_gen))

[[ 3.21931635e-03 -6.73645054e-04  8.65154598e-03 ... -1.29926033e-02
  -9.33028932e-03 -1.34207197e-02]
 [ 2.68387691e-03 -7.36425956e-04  1.20886223e-02 ... -1.97474279e-03
   1.25340658e-02  3.10030774e-03]
 [ 1.83457838e-02 -2.27686916e-03  1.56622476e-01 ...  4.14038861e-02
  -7.98423892e-02 -8.06254233e-03]
 ...
 [ 2.17170455e-04 -5.02625441e-05  1.18880020e-03 ... -7.16678704e-04
  -8.28912443e-05  5.48399087e-03]
 [ 3.47115700e-03 -4.34124885e-04  4.76333215e-03 ... -4.93589395e-03
  -7.07182183e-03  1.61380821e-02]
 [ 2.54227742e-03 -8.43560494e-04  3.67024821e-03 ... -4.46934720e-03
  -9.90845001e-03  2.37969378e-02]]


Попытка применить НС

In [ ]:
import torch
import torch.nn as nn

trainX=torch.FloatTensor(X_sparse_tsvd)
trainY=torch.FloatTensor(y_tr)

class network(nn.Module):
  def __init__(self,n_input, n_hidden):
    super(network,self).__init__()
    self.fc1=nn.Linear(n_input, n_hidden)
    self.fc2=nn.Linear(n_hidden, n_hidden)
    self.fc3=nn.Linear(n_hidden, 1)
    self.act1=nn.ReLU()
    self.act2=nn.ReLU()
    self.sm=nn.Sigmoid()
    #self.loss=nn.CrossEntropyLoss()
    self.loss=nn.BCEWithLogitsLoss()

  def forward(self,x):
    x=self.fc1(x)
    x=self.act1(x)
    x=self.fc2(x)
    x=self.act2(x)
    x=self.fc3(x)
    return x

  def fit(self,trainX,trainY, epochs=100, batch_size=20):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-2,betas=(0.9, 0.99))#, betas=(0.9, 0.99)
    for epoch in range(epochs):
      idx=np.random.permutation(trainX.shape[0]) #массив индексов зад. длины
      for i in range(0,trainX.shape[0],batch_size):
        optimizer.zero_grad()
        batch_ind=idx[i:i+batch_size]
        x_batch=trainX[batch_ind]
        y_batch=trainY[batch_ind]

        y_pred=self.forward(x_batch).ravel()
        loss_val=self.loss(y_pred, y_batch)
        loss_val.backward()
        optimizer.step()

  def predict(self, x):
    x=self.forward(x)
    x=self.sm(x)
    return x

#Net=network(trainX.shape[1],10)
#Net.fit(trainX,trainY)
n=1
acc=0
for i in range(trainY.shape[0]):
  preds=(Net.predict(trainX[i]).detach().numpy())
  preds=np.where(preds>0.4,1,0)
  acc += ((preds==trainY[i].numpy())+y_pred_url[i])/2
print("train accuracy: ",acc/trainY.shape[0])

train accuracy:  [0.90200401]


In [ ]:
from  gensim.models.KeyedVectors import load_word2vec_format
def load_word2vec():
    word2vecDict = load_word2vec_format(
        '../input/word2vec-google/GoogleNews-vectors-negative300.bin',
        binary=True, unicode_errors='ignore')
    embeddings_index = dict()
    for word in word2vecDict.wv.vocab:
        embeddings_index[word] = word2vecDict.word_vec(word)

    return embeddings_index

In [ ]:
import gensim
model_path = '/content/model.hdf5'
model = gensim.models.KeyedVectors.load_word2vec_format(model_path)

def vectorize_sentences(sentences):
    vectorized_sentences = []
    for sentence in sentences:
        sentence_vector = []
        words = sentence.split()
        for word in words:
            try:
                word_vector = model[word]
                sentence_vector.append(word_vector)
            except KeyError:
                # Если слово отсутствует в словаре, пропускаем его
                pass
        vectorized_sentences.append(np.mean(sentence_vector, axis=0))  # Усредняем векторы слов для получения одного вектора предложения
    return np.array(vectorized_sentences)